In [1]:
!pip install -q sentence-transformers
!pip install -q transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.3 MB/s eta 0:00:00


In [2]:
import os

# Load and combine all clean .txt files
def load_all_text_files(folder="/content"):
    all_text = ""
    for file in os.listdir(folder):
        if file.endswith("_clean.txt"):
            with open(os.path.join(folder, file), "r", encoding="utf-8") as f:
                all_text += f.read() + "\n\n"
    return all_text

# Chunk the combined text into ~100-word pieces
def chunk_text(text, max_words=100):
    chunks = []
    current_chunk = []
    for sentence in text.split('.'):
        words = sentence.strip().split()
        if not words:
            continue
        current_chunk.extend(words)
        if len(current_chunk) >= max_words:
            chunks.append(' '.join(current_chunk))
            current_chunk = []
    if current_chunk:
        chunks.append(' '.join(current_chunk))
    return chunks

# Run the loading and chunking
full_text = load_all_text_files("/content")
chunks = chunk_text(full_text, max_words=100)

print("Total chunks created:", len(chunks))
print("Sample chunk:\n", chunks[0][:300])


Total chunks created: 36
Sample chunk:
 [SECTION: VOX_LANGUAGE] Lost In Translation: Language Barriers at IITK – Vox Populi 15 5% of the respondents have little to no comfort with the Hindi language The inability to interact fluently in Hindi unfurls another set of issues A common problem mentioned by the respondents is that the mess work


In [3]:
from sentence_transformers import SentenceTransformer
import pickle

# Load embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2')
print("Embedder model loaded.")

# Generate embeddings for all chunks
embeddings = embedder.encode(chunks, convert_to_tensor=True)
print("Embeddings created.")

# Save chunks and embeddings for reuse
with open("rag_chunks.pkl", "wb") as f:
    pickle.dump(chunks, f)

with open("rag_embeddings.pkl", "wb") as f:
    pickle.dump(embeddings, f)

print("Chunks and embeddings saved to rag_chunks.pkl and rag_embeddings.pkl")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedder model loaded.
Embeddings created.
Chunks and embeddings saved to rag_chunks.pkl and rag_embeddings.pkl


In [50]:
from transformers import pipeline
from sentence_transformers import util
import torch

# Load saved chunks and embeddings
with open("rag_chunks.pkl", "rb") as f:
    chunks = pickle.load(f)
with open("rag_embeddings.pkl", "rb") as f:
    embeddings = pickle.load(f)

# Load FLAN-T5 generator
generator = pipeline("text2text-generation", model="google/flan-t5-base")
print("FLAN-T5 model loaded.")

# RAG-style answer function
def answer_question(question, top_k=2):
    with open("rag_chunks.pkl", "rb") as f:
        chunks = pickle.load(f)
    with open("rag_embeddings.pkl", "rb") as f:
        embeddings = pickle.load(f)

    question_embedding = embedder.encode(question, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, embeddings, top_k=top_k)[0]
    top_chunks = [chunks[hit['corpus_id']] for hit in hits]


    # Limit combined context to ~400 words
    context = " ".join(top_chunks)
    context_trimmed = " ".join(context.split()[:400])

    top_score = hits[0]['score']
    # Convert score to label
    if top_score >= 0.7:
        confidence_label = "High Confidence"
    elif top_score >= 0.5:
        confidence_label = "Moderate Confidence"
    else:
        confidence_label = "Low Confidence"


    #prompt = f"Answer this question using the context below:\n\nQuestion: {question}\n\nContext: {context_trimmed}"
    prompt = f"""Answer the following question in 2-3 detailed paragraphs using the context below.
    Question: {question}
    Context: {context_trimmed}"""

    result = generator(prompt, max_new_tokens=150)[0]['generated_text']
    return result, context_trimmed, round(top_score, 3), confidence_label



Device set to use cpu


FLAN-T5 model loaded.


In [53]:
question = "Tell me about language barriers in IITK"
answer, context_used, score, confidence = answer_question(question)

print("Answer:\n", answer)
print("\nRelevance Score:", score)
print("Answer Confidence:", confidence)


Answer:
 A common problem mentioned by the respondents is that the mess workers, maintenance staff, etc , speak only in Hindi Several also said they have difficulty interacting with new people and participating in campus activities as they are usually conducted in Hindi Sharah PS, a Malayalam-speaking UG Y22, shares her experience of how she has to take a Hindi-fluent friend with her whenever she has to interact with shopkeepers In conversation with Mudit Agarwal (Outgoing GenSec) With his participation in three consecutive Techmeets, Mudit explains how IITK has grown to have students interested in diverse domains and fields Mudit also talked about how the

Relevance Score: 0.629
Answer Confidence: Moderate Confidence


In [54]:
from google.colab import files

files.download("rag_chunks.pkl")
files.download("rag_embeddings.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>